In [184]:
using Plots
using JLD
using StatsPlots
using LaTeXStrings
using Statistics

include("../src/FCSeqTools.jl");
include("../src/functions.jl");

model1 = "EAA_(0.05_0.01_nb1)"
model2 = "EAA_(0.05_0.01_nb2)" #"EAA_bmDCA"
folder_name = model1*"_vs_"*model2
alignment = "CM_alignment"

# Se è un modello di Francesco Zamponi ######################################
#model_zamponi = model2[1:3]
#############################################################################

rm(joinpath("figures/"* alignment * "/comparison_models",folder_name), force=true, recursive=true)
mkdir(joinpath("figures/"* alignment * "/comparison_models",folder_name))
path = joinpath("figures/"* alignment * "/comparison_models",folder_name);

L, q = 96, 21;

# Compare Couplings

In [185]:
method = "cumulative"
fraction = 0.3
stop = 0.9
α = 0.05 # pseudo count 
init_pseudo_count = 0.01


# Choose Notebooks ##################################
test = "main_test"
nbook1 = "1"
nbook2 = "2"

folder1 = "../training/" * alignment * "/" * method * "/" * test * "/"
folder2 = "../training/" * alignment * "/" * method * "/" * test * "/"
######################################################

path1 = folder1 *method*string(fraction)*"_stop="*string(stop) *"_reg="*string(α) *"_nb"*nbook1
path2 = folder2 *method*string(fraction)*"_stop="*string(stop) *"_reg="*string(α)*"_nb"*nbook2

h1 = JLD.load(path1* "/h.jld")["data"]
Jij1 = JLD.load(path1* "/Jij.jld")["data"];
iteration1 = open(path1* "/iterations.txt", "r")
data1 = parse.(Float64, split(readlines(iteration1)[end], " ")[1:end-1])
edge_n1, element_n1 = data1[6], data1[7]

# Se è un modello dei nostri #################################################

h2 = JLD.load(path2* "/h.jld")["data"]
Jij2 = JLD.load(path2* "/Jij.jld")["data"];
iteration2 = open(path2* "/iterations.txt", "r");
data2 = parse.(Float64, split(readlines(iteration2)[end], " ")[1:end-1])
edge_n2, element_n2 = data2[6], data2[7]

#############################################################################

# Se è un modello di Francesco Zamponi ######################################
"""
filepath2 = "../saved_models/Francesco_Zamponi/Parameters_CM_"* model_zamponi *"-PCD40.dat";
Jij2, h2 = read_model(filepath2);
"""
#############################################################################

Nij_tot = L * (L - 1) /2
Nij_ab_tot = Nij_tot * q^2;

#### Scatter Plot of all the Jijs 

In [186]:
n_element_plot = 50_000
Jij1_to_plot = zeros(n_element_plot);
Jij2_to_plot = zeros(n_element_plot);

for i in 1:n_element_plot
    j = rand(1:length(Jij1))
    Jij1_to_plot[i] = Jij1[j]
    Jij2_to_plot[i] = Jij2[j]
end

scatter(Jij1_to_plot, Jij2_to_plot, markershape=:circle, label=:false, xlabel="Jij(a,b) ("*model1*")", ylabel="Jij(a,b) ("*model2*")", title="Couplings nb.1 vs nb.2", alpha=0.2)
savefig(joinpath(path,"All_the_Jij1_vs_Jij2.png"));

### Count Jijs in common and plot the distributions

In [187]:
Jij_plot_1 = Jij1
Jij_plot_2 = Jij2

n_ij, n_ij_ab, n_01, n_10, n_00, Jij_common1, Jij_common2 = count_common_Jij(Jij_plot_1, Jij_plot_2);

frac_ij_tot = round(n_ij/Nij_tot * 100, digits=2)
frac_ijab_tot = round(n_ij_ab/Nij_ab_tot * 100, digits=2)

frac_ij_1 = round(n_ij/edge_n1 * 100, digits=2)
frac_ijab_1 = round(n_ij_ab/element_n1 * 100, digits=2)

frac_ij_2 = round(n_ij/edge_n2 * 100, digits=2)
#frac_ijab_2 = round(n_ij_ab/element_n2 * 100, digits=2)

complex_1 = round(count(!iszero, Jij_plot_2)/Nij_ab_tot*100, digits=2)
complex_2 = round(count(!iszero, Jij_plot_2)/Nij_ab_tot*100, digits=2)


# Se è un modello di Francesco Zamponi ############################################################################

J2 = zeros(L, L)
for i ∈ 1:L
       for j ∈ 1:L
              J2[i, j] = sum(Jij2[i, j, :])
       end
end
frac_ij_2 = round(n_ij/count(!iszero, J2) * 100, digits=1)
frac_ijab_2 = round(n_ij_ab/count(!iszero, Jij_plot_2) * 100, digits=1)

##################################################################################################################


### WRITE ON FILE ################################################################################################################################################
filename = "count_common_Jij"
open(joinpath(path, filename), "w") do f  
       println("Complexity model 1: ",complex_1,"%;    Complexity model 2: ",complex_1, "%")
       write(f,"Complexity model 1: $(complex_1)%;    Complexity model 2:    $(complex_1)%;   ")

       println("\nCommon Edges: \n",frac_ij_tot,"% of the total (L^2);   ", frac_ij_1, "% of the nonzeros of model 2;   ", frac_ij_2, "% of the nonzeros of model 2")
       write(f,"\nCommon Edges: \n $(frac_ij_tot)% of the total;   $(frac_ij_1)% of the first model;   $(frac_ij_2)% of the second model")

       println("\nCommon Elements: \n",frac_ijab_tot,"% of the total (L^2*q^2);   ",  frac_ijab_1, "% of the nonzeros in model 1;   ", frac_ijab_2, "% of the nonzeros of model 2")
       write(f,"\nCommon Elements: \n $(frac_ijab_tot)% of the total;   $(frac_ijab_1)% of the first model;   $(frac_ijab_2)% of the second model")

       println("\nNumber of cases where  J_1 == 0  &&  J_2 != 0: ", n_01)
       write(f,"\nNumber of cases where  J_1 == 0  &&  J_2 != 0: $n_01")

       println("\nNumber of cases where  J_1 != 0  &&  J_2 == 0: ", n_10)
       write(f,"\nNumber of cases where  J_1 != 0  &&  J_2 == 0: $n_10")

       println("\nNumber of cases where  J_1 == 0  &&   J_2 == 0: ", n_00)
       write(f,"\nNumber of cases where  J_1 == 0  &&   J_2 == 0: $n_00")

       println("\nNumber of cases where  J_1 != 0  &&  J_2 != 0: ", n_ij_ab)
       write(f,"\nNumber of cases where  J_1 != 0  &&  J_2 != 0: $n_ij_ab")
end
##################################################################################################################################################################

# Plot Scatter & Distributions #####################################################################################
x, y = Jij_common1[Jij_common1.!= 0.0], Jij_common2[Jij_common2.!= 0.0]
x_label, y_label = "Jij(a,b)"*model1, "Jij(a,b)"*model2 # (EAA 0.5)", "Jij(a,b)"*model_zamponi
title = "Common Couplings: "* L"J_{ij}(a,b)"


plot(  histogram(x, nbins=100, alpha=0.5, orientation=:vertical,label=:false, xticks=[], ylabel="counts", title = title),   
       plot(framestyle = :none),
       scatter(x, y, markershape=:circle, label=:false, xlabel=x_label, ylabel=y_label, alpha=0.2),
       histogram(y, nbins=100, alpha=0.5, orientation=:horizontal, label=:false, yticks=[], xlabel="counts"),
       link = :both, 
       size=(1000, 1000), 
       spacing=0.1, 
       )
savefig(joinpath(path,"Common_Jij1_vs_Jij2.png"));
##################################################################################################################

Complexity model 1: 1.93%;    Complexity model 2: 1.93%

Common Edges: 
34.08% of the total (L^2);   90.4% of the nonzeros of model 2;   88.5% of the nonzeros of model 2

Common Elements: 
0.74% of the total (L^2*q^2);   39.22% of the nonzeros in model 1;   38.4% of the nonzeros of model 2

Number of cases where  J_1 == 0  &&  J_2 != 0: 23903

Number of cases where  J_1 != 0  &&  J_2 == 0: 23079

Number of cases where  J_1 == 0  &&   J_2 == 0: 1991422

Number of cases where  J_1 != 0  &&  J_2 != 0: 14892


#### Count and plot Jijs NOT in common

In [188]:
J_diff1 = Jij1[(Jij1.!= 0).*(Jij2.== 0)] 
J_diff2 = Jij2[(Jij2.!= 0).*(Jij1.== 0)]; 

In [189]:
histogram(J_diff1, bins = 100, title = "Jij non in common", xlabel="values", ylabel="counts", label=:false)
savefig(joinpath(path,"Jij1_NOT_in_common_with_Jij2.png"));

In [190]:
histogram(J_diff2, bins = 100, title = "Jij non in common", xlabel="values", ylabel="counts", label=:false)
savefig(joinpath(path,"Jij2_NOT_in_common_with_Jij2.png"));

#### Common Jijs vs Score/Iterations


In [191]:
file_path1 = joinpath(path1, "example_output.txt");
file_path2 = joinpath(path2, "example_output.txt");


edges1, elements1, scores1 = read_example_output(file_path1);
edges2, elements2, scores2 = read_example_output(file_path2);

## Common Edges/Elements vs Cij score

In [192]:
n_edges, n_elements, n_edges_x, n_edges_y, n_elements_x, n_elements_y = compare_edges(edges1, elements1, edges2, elements2);

In [ ]:
scatter(scores_x, n_elements./n_elements_x * 100, marker=:circle, markersize=3, label="elements", legend=:right)
scatter!(scores_x, n_edges./n_edges_x * 100, marker=:circle, markersize=3, label="edges")
plot!(scores_x, 90*ones(length(scores_x)), label="90%")
plot!(scores_x, 50*ones(length(scores_x)), label="50%")
xlabel!(L"C_{ij}" * " score")
ylabel!("%")
title!("Percentage of non-zero parameters in both models")
savefig(joinpath(path,"number_shared_Jij_vs_Cij_score.png"));

In [ ]:
scatter(1:length(n_elements), n_elements./n_elements_x * 100, marker=:circle, markersize=3, label="elements", legend=:right)
scatter!(1:length(n_elements), n_edges./n_edges_x * 100, marker=:circle, markersize=3, label="edges")
plot!(1:length(n_elements), 90*ones(length(n_elements)), label="90%")
plot!(1:length(n_elements), 50*ones(length(n_elements)), label="50%")
xlabel!("E.A.A. iteration")
ylabel!("%")
title!("Percentage of non-zero parameters in both models")
savefig(joinpath(path,"number_shared_Jij_vs_iterations.png"));